# Urban Hydrological Modeling

In this section, we will look at urban hydrological modeling approaches and how to set up models from scratch. We will **not** look at commercially or open-source models that are often used for urban water modeling, as these models frequently simplify the local hydrology.

## Urban Mass-Balance Modeling

One of the most common and most powerful ways of modeling the urban hydrological cycle is using the mass-balance method (see {numref}`urban_water_cycle`). This method, which relies on the principle of mass conservation (often assuming constant water density and therefore translating to volume conservation), is expressed through a single equation:

$$
\frac{dS}{dt} = Q_{in} - Q_{out}
$$ (eq:mass_balance_modeling)

with:

|  |   |  |  
| --- | --- | --- |  
| $dS$ | Change in stored volume in a particular part of the hydrological cycle | $L^3$ |  
| $dt$ | Time interval | $T$ |  
| $Q_{in}$ | Inward flux to the given storage | $\frac{L^3}{T}$ |  
| $Q_{out}$ | Outward flux from the given storage | $\frac{L^3}{T}$ |  

The equation can be applied to all storages and fluxes shown in {numref}`UWB`. To create a hydrological model, the various mass balances are integrated into a single model, such that the outflow $Q_{out}$ of one storage is the inflow $Q_{in}$ of the next.

The model can be applied either as a dynamic model, calculating the fluxes at every timestep $t$, or as a cumulative model, which estimates the fluxes over a longer period of time. The latter is more computationally efficient but may miss some of the faster dynamics that occur in the urban environment. With this in mind, we will mainly focus on the dynamic implementation of the mass-balance modeling approach while also providing some examples of the cumulative model implementation.

The temporal and spatial scales should be considered before setting up the model. This decision depends on the objectives of the model and the properties of the system. These considerations will be examined per sub-section of the model. Below, we will analyze the mass balance for each section and highlight the spatio-temporal considerations. In this implementation, we will consider the atmosphere, the open water system, and the deep groundwater as boundary conditions.

The key concept of urban hydrology (compared to "natural hydrology") is that all the storages have *storage capacities*, and all the fluxes have *maximum flux rates*. The fluxes out of the system are often dependent on whether the storage capacity or (one of) the maximum flux rates has been reached. The capacities and maximum flux rates, combined with the conditional statements, are highlighted per storage unit below.


### Impervious Land Surface Storage  

- **Storage Capacity**: Water can be stored on the impervious land as depression or wetting loss storage. On impervious lands, these can be considered as one, usually with a storage capacity between **0.5-4 mm**. This storage is in the characteristic puddles, and the wetting depth is spread throughout the impervious areas.  

- **Maximum Fluxes**: The maximum outflow from the land surface storage is limited when it is considered the flux through a gully pot or roof gully (both inward fluxes to the sewer system), having a maximum rate of **50 mm/hr** and **40 mm/hr**, respectively, for a typical Dutch system.  

The mass-balance for the impervious land surface storage therefore becomes:  

$$
\frac{dS_{s,imp}}{dt} = P - Q_{s} - Q_{o} - Q_{q} - E_{t}
$$ (eq:impervious_mass_balance)

The precipitation $P$ is considered an input parameter and is therefore not quantified by the model. Similarly, the evaporative part $E_{t}$ tends to be considered an input parameter, but it can, however, be quantified as part of the model {numref}`urban_evap_model`.  

The other outward fluxes $Q_{s}$, $Q_{o}$, and $Q_{q}$ depend on the specific configuration of the area. The latter two are overland flow and are not considered to have a maximum rate. $Q_{s}$ is a consequence of designed infrastructure and therefore does have limits. In a typical Dutch system, the flow through a gully pot (from the road into the urban drainage system) has a maximum rate of around **50 mm/hr**, and roof connections similarly have a maximum rate of around **40 mm/hr**.  

Note the units: the flow rates are normalized against the area that they are designed to drain. All three fluxes are only activated if the storage capacity on the impervious area is exceeded (this is a model assumption). Their relative partitioning depends on the urban layout. Therefore, for all cases, the outward flux can be calculated through:

$$
Q_{t}= 
\begin{cases}
    \min(Q_{i,max}, c_{i} \cdot (P - E_{t})) & \text{if} \text{ } S_{s,imp} \geq S_{s,imp, max} \\
    0              & \text{otherwise} &
\end{cases}
$$ (Q_t_cases)

with:

|  |   |  |  
| --- | --- | --- |  
| $Q_{t}$ | Relevant outflow at timestep $t$ | $\frac{L^3}{T}$ |  
| $P$ | Precipitation | $\frac{L^3}{T}$ |  
| $E_t$ | Evaporation from the area | $\frac{L^3}{T}$ |  
| $c_{i}$ | Coefficient determining the amount of flow going through the $i^{th}$ flux | [-] |  
| $Q_{i,max}$ | Maximum capacity of the relevant flux | $\frac{L^3}{T}$ | 

When considering the temporal scale, the mass-balance approach should be used for relatively short time periods, preferably at a 5-15 min resolution. If the maximum capacity of the outflow fluxes is breached, this is likely to happen during a short period of time. If the rainfall and evaporative fluxes are then considered at a higher resolution, an over- and underestimation in the outward fluxes can be expected.  

The spatial scale depends entirely on the objective of the model. If the quantification or analysis of local issues is the modeling objective, then a higher spatial resolution should be used (i.e., the modeled area should be smaller per impervious storage section in the model).  

### Pervious Land Surface Storage

- **Storage Capacity**: Water can be stored on pervious land as depression or wetting loss storage. Contrary to the modeling approach for impervious surfaces, these are considered separately because of their different outward fluxes. The depression storage is significantly larger, with a typical size of **1-3 mm**. The wetting loss is smaller, usually in the range of **0-1 mm**.  

- **Maximum Fluxes**:  <span style="color: red;">incomplete</span>


(urban_drain_net_stor)=
### Urban Drainage Network Storage  

- **Storage Capacity**: Characteristic of Dutch urban drainage networks is their relatively large storage capacity, usually in the range of **6-11 mm**. This is due to the flatness of typical Dutch cities: large portions of the network require pumping to transport water to the treatment plant. To avoid unnecessary overflows into surface water, a large buffer capacity must be realized.  

- **Maximum Fluxes**: The urban drainage network comprises solely designed components. As such, most of the inward and outward fluxes have defined maximum fluxes. Notably:  
  - The **pumping into the wastewater treatment plant** (part of $Q_{d}$) has a design capacity of between **0.4 mm/hr** and **1.1 mm/hr**.  
  - The **drainage outflows** (another part of $Q_{d}$), which are uncontrolled discharges through weir structures, have a capacity of around **25 mm/hr**.  

The mass balance for the urban drainage network becomes the following:

$$
\frac{dS_{sw}}{dt} = Q_{s} + Q_{p} + Q_{c} + I_{e} - Q_{d} - I_{d}
$$ (eq:urban_drainage_mass_balance)

where:  

- $dS_{sw}$ is the change in volume stored in the drainage network,  
- $Q_{s}$ ($\frac{L^3}{T}$) is the runoff from impervious areas (roofs and roads, through gully pots and roof gullies, the outward flux $Q_{s}$ from the impervious calculation),  
- $Q_{p}$ ($\frac{L^3}{T}$) is the runoff from pervious areas,  
- $Q_{c}$ ($\frac{L^3}{T}$) is the wastewater production into the combined sewer (if it is a separate sewer network, this inflow is 0),  
- $I_{e}$ ($\frac{L^3}{T}$) is the groundwater inflow into the UDS,  
- $Q_{d}$ ($\frac{L^3}{T}$) is the outward flux consisting of three components:  
  1. Influent to the treatment plant  
  2. Combined sewer overflows  
  3. Flooding  
- $I_{d}$ ($\frac{L^3}{T}$) is the outward flux into the groundwater, named exfiltration. This flux is only relevant if the groundwater table is below the invert elevation of the sewer network.  

The two fluxes $I_{e}$ and $I_{d}$ can be significant when considering the dry weather flow conditions (and therefore also the annual loading) of the UDS. They are quantified at the UDS network level and are given as a constant infiltration or exfiltration rate. If the groundwater level is highly variable, the infiltration and exfiltration change based on the groundwater level. Within the mass-balance modeling approach, this is not considered explicitly. When a rain event occurs, the inflows $Q_{s}$ and $Q_{p}$ and outward flux $Q_{d}$ tend to be much larger.  

As mentioned, the outward flux, $Q_{d}$ consists of three components:

$$
Q_{d} = Q_{wwtp} + Q_{cso} + Q_{flood}
$$ (eq:outward_flux_components)

with:

|   |   |   |
| --- | --- | --- |
| $Q_{wwtp}$ | Outward flux towards the treatment plant | $\frac{L^3}{T}$ |  
| $Q_{cso}$ | Outward flux through the combined sewer overflow (CSO) | $\frac{L^3}{T}$ |  
| $Q_{flood}$ | Outward flux due to flooding | $\frac{L^3}{T}$ |  

$Q_{flood}$ should technically be directed back to the impervious land surface storage, as this is where the flooded water is temporarily "stored." However, in hydrological modeling, the volume of the flooded water is the number that modelers are interested in. Therefore, diverting it straight into a surface water storage node is sufficient in urban hydrological modeling.  

The individual components of $Q_{d}$ can be modeled as follows:

$$
Q_{wwtp} = \min(Q_{pump}, \frac{S_{sw}}{dt})
$$ (eq:wwtp_flow)

with:

|   |   |   |
| --- | --- | --- |
| $Q_{pump}$ | Maximum pumping capacity of the wastewater treatment influent pumps | $\frac{L^3}{T}$ |  
| $S_{sw}$ | Storage volume in the UDS | $L^3$ |  
| $dt$ | Timestep | $T$ |  

This equation reflects that the pump can only pump the equivalent pumping capacity *or* the amount of water that is stored in the UDS. If the latter is smaller than the pumping capacity, less water will be discharged into the WWTP than possible based on the installed pump over-capacity (POC).

$$
Q_{CSO} = 
\begin{cases}
    \min(Q_{cso, max}, Q_{s} + Q_{p} + Q_{c} - \frac{S_{sw,max} - S_{sw}}{dt} + I_{e} - I_{d}), & \text{if} \text{ } S_{sw} + Q_{total,in} - Q_{total,out} \geq S_{sw,max} \\
    0, & \text{otherwise}
\end{cases}
$$ (eq:cso_outflow)

with:

|   |   |    |
| --- | --- | --- |
| $Q_{cso, max}$ | Maximum CSO rate | $\frac{L^3}{T}$ |  
| $\frac{S_{sw,max} - S_{sw}}{dt}$ | Remaining storage space in the UDS | $\frac{L^3}{T}$ |  
| $Q_{total,in}$ | Sum of all inward fluxes | $\frac{L^3}{T}$ |  
| $Q_{total,out}$ | Sum of all outward fluxes | $\frac{L^3}{T}$ |  
| $S_{sw, max}$ | Maximum storage capacity of the UDS | $L^3$ |  

What this equation represents is that the CSO is only activated if the storage capacity of the UDS is exceeded, i.e., the total inward fluxes minus the outward fluxes are more than the remaining capacity.

The equation for $Q_{flood}$ takes a similar form as $Q_{cso}$:

$$
Q_{flood}= 
\begin{cases}
    Q_{s} + Q_{p} + Q_{c} - \frac{S_{sw,max}-S_{sw}}{dt} + I_{e} - I_{d} - Q_{cso} & \text{if} \text{ } S_{sw} + Q_{total,in} - Q_{total,out} \geq S_{sw, max}\\
    0              & \text{otherwise}
\end{cases}
$$ (eq:flooding_equation)

The flooding, following the hydrological modeling approach, can only occur if the entire sewer network is filled ($Q_{sw} > Q_{sw, max}$) *and* $Q_{cso}$ is equal to the maximum rate. Such flooding events are relatively rare, as UDS are designed to a return period of 1-in-2 years or 1-in-5 years, depending on the sensitivity of the flooded area (considering the presence of emergency services, among others).

The urban drainage network storage module is modeled at either the full network level or at a *sewer district* level. This is a section of the sewer network that is hydraulically separated from the rest of the sewer network through either a pump or a controlled discharge point. The spatial scale of the UDS usually drives the spatial scale of the other components of the model, as this is the most common discharge or interactive point.  

Considering the temporal scale, similar to runoff, it depends on the objective of the model. CSO and flooding events tend to be relatively short in occurrence, dominated by the peak intensity rainfall that occurs for 5-20 minutes in the Netherlands. Considering a larger timescale might mean that the model "misses" these extreme events.

### Soil Storage

- **Storage Capacity**: The amount of water that can be stored depends on the soil porosity and permeability. This varies between soil types, with clays having a larger storage capacity compared to sands.  

- **Maximum Fluxes**: The maximum inward and outward fluxes depend on the hydraulic conductivity of the soil. Usually, there are several layers in the subsurface, leading to different hydraulic conductivities. The lowest conductivity is the limiting factor when considering the outward flux, and the top layer is the limiting factor when considering the inward flux into the soil. Although the native soil (the soil type that was there before construction) is often still present, soils can be refilled in order to promote faster infiltration. This is often the case for bioswales and other BGIs, where there is a minimum infiltration rate that needs to be reached, usually between **1 cm/hr** and **2 cm/hr** as a minimum rate.  

The mass balance for the soil storage becomes:

$$
\frac{dS_{u}}{dt} = I_{s} + I_{d} + C + Q_{i} - E_{T} - E_{S} - R - I_{e}
$$ (eq:soil_mass_balance)

with:

|   |   |  |  
| --- | --- | --- |  
| $C$ | Groundwater seeping into the soil layer | $\frac{L^3}{T}$ |  
| $R$ | Deep infiltration into the groundwater | $\frac{L^3}{T}$ |  
| $Q_{i}$ | Leakage from the drinking water network | $\frac{L^3}{T}$ |  
| $E_{T}$ | Transpiration from vegetation on the pervious surface | $\frac{L^3}{T}$ |  
| $E_{S}$ | Soil evaporation from the pervious surfaces | $\frac{L^3}{T}$ | 

Similar to $I_{d}$ and $I_{e}$, $Q_{in}$, $C$, and $R$ are often considered as constant inward or outward fluxes. More commonly, the latter two fluxes are combined into a *net groundwater flux*, which can be either positive or negative depending on the specific time scale and location. $E_{T}$ and $E_{S}$ are meteorologically dependent and are detailed in the <span style="color: red;">energy balance modeling part of this chapter</span>.  

The infiltration from the pervious ground, $I_{s}$, can be modeled using Richards Equation:

$$
\frac{d\theta}{dt} = k(\theta) \cdot dh
$$ (eq:richards_equation)

with:

|   |   |   |  
| --- | --- | --- |  
| $\theta$ | Soil moisture content | [-] |  
| $k(\theta)$ | Hydraulic conductivity corresponding to the soil moisture content | $\frac{L}{T}$ |  
| $dh$ | Head of water driving the horizontal flow | $L$ |  

Although Richards Equation (or other implementations, like Horton's equation, Green-Ampt, and the curve number method) can accurately model the infiltration rate, using it within a mass-balance hydrological model can be computationally complex and contain a level of detail that is beyond the scope of a mass-balance model.  

For this reason, the variable hydraulic conductivity ($k(\theta)$) is often replaced with the constant saturated hydraulic conductivity. Although this can lead to inaccuracies in the model, the changes in infiltration rates for a mass-balance model are modest. Additionally, the head driving the flow is assumed to be atmospheric and constant, leading to the simplified equation:

$$
I_{s} = \min(S_{s,per} \cdot dt, k \cdot A)
$$ (eq:simplified_infiltration)

with:

|   |   |   |  
| --- | --- | --- |  
| $S_{s,per}$ | Amount of water stored on the pervious surface available for infiltration | $L^3$ |  
| $dt$ | Modeling timestep | $T$ |  
| $k$ | Saturated hydraulic conductivity | $\frac{L}{T}$ |  
| $A$ | Impervious area | $L^2$ |  

This modeling approach allows infiltration to occur only when water is available for infiltration.

The soil layer is highly heterogeneous, making it challenging to combine into a single storage unit. Spatial considerations are therefore driven by the heterogeneity of the soil in the modeled area. Various soil storages can be added to a single model, with fluxes transporting water between the storages. This depends on the target resolution of the model. More often than not, limited information is available concerning the soil composition, making accurate modeling more difficult.  

The temporal scale is less stringent for the soil layer. The (deep) infiltration processes are relatively slow compared to the previously described processes, which means that a lower temporal resolution (hourly) should be sufficient.  

### Surface Water Storage  

- **Storage Capacity**: Urban canals are designed to buffer a significant portion of rainwater to prevent flooding from the open water system. To this end, open water systems have a storage capacity of around **30 mm**. Usually, larger areas discharge into a single canal system, meaning that in absolute terms (**m³**), this is the largest storage unit in the urban hydrological cycle.  

- **Maximum Fluxes**: The storage capacity is maintained through a system of pumps and sluice gates (*NL: kunstwerken*). The surface water pumps (*NL: oppervlaktewater gemalen*), usually located at the boundary of the system, have a capacity of around **0.5 mm/hr**. The inward fluxes are also limited but have been discussed in {numref}`urban_drain_net_stor`.

The mass balance for surface water becomes:

$$
\frac{dS_{sw}}{dt} = P + Q_{q} + Q_{d} + Q_{g} - E_{o} - Q - L
$$ (eq:surface_water_mass_balance)

with: 

|   |   |   |  
| --- | --- | --- |  
| $dS_{sw}$ | Storage capacity of the surface water system | $L^3$ |  
| $E_{o}$ | Free water evaporation | $\frac{L^3}{T}$ |  
| $Q$ | Water being drained out of the system | $\frac{L^3}{T}$ |  
| $Q_{g}$ | Inward flux from the groundwater | $\frac{L^3}{T}$ |  
| $L$ | Infiltration of water into the ground | $\frac{L^3}{T}$ |  

The outflow $Q$ depends on the operative rules used in the area. Proactive water level management, as deployed in Rotterdam, reduces the water level if an increased influx (rainfall) is predicted. This allows the capacity of the surface water to be increased by the pumping capacity (usually around **0.5 mm/hr**). The quicker the decision is made, the more storage capacity can be realized. To model the outflow, the operational rules for surface water management must be known.  

Similar to soil storage, the two fluxes $L$ and $Q_{q}$ are often combined into a *net groundwater contribution* value, which is considered constant throughout the modeling range. This net groundwater contribution can be **positive** (if groundwater levels are high enough to discharge into the river) or **negative** (when the river recharges the groundwater).  

Considering the temporal and spatial scale, surface water processes are relatively slow compared to Urban Drainage Network Storage-related processes. **Hourly modeling intervals** are often sufficient to capture the most important dynamics. On a spatial scale, the delimitation of the surface water system is often defined by the presence of *kunstwerken* (hydraulic structures): a hydraulically separated section is considered a single storage unit.  

```{Note}
Within the presented modeling framework, BGIs are not explicitly integrated. However, BGI components can be added in the same mass-balance form as the other components. For example, a **bioswale** can be explicitly modeled as an intermediate storage between the impervious/pervious land surface storages and the sewer network. The mass-balance modeling approach allows for a highly flexible framework that can be adapted to specific modeling requirements.
 ```


## Alternative Method: Runoff Modeling  

The mass-balance approach is a very powerful tool for modeling the entire urban water balance. However, in the design of urban environments, we are often only interested in the runoff into and out of the urban drainage network. Setting up a detailed mass-balance model would be too cumbersome.  

To this end, the next section describes the *Rational Method*, which can be used to model the outflows of a catchment. The term *catchment* can refer to various spatial scales and flow types. The Rational Method can be applied to:  

- A **catchment draining into a gully pot** (typically **10s–100s of m²**),  
- The **outflow of a larger upstream area** (typically **hectares**),  
- The **flow through a pipe network** such as an urban drainage system (**draining 10s of hectares**).  

### Rational Method  

The Rational Method is a single equation that can be used to calculate the *maximum outflow* from a catchment over a given time period. The equation takes the form:  

$$
Q_{max} = c_{r} \cdot i \cdot A
$$ (eq:rational_method)

with:

|   |   |   |  
| --- | --- | --- |  
| $Q_{max}$ | Peak outflow of a catchment for the given time period | $\frac{L^3}{T}$ |  
| $c_{r}$ | Runoff coefficient | [-] |  
| $i$ | Rainfall for the given time period | $\frac{L}{T}$ |  
| $A$ | Connected area | $L^2$ |  

As you can see, this is a straightforward equation where the peak outflow can be calculated. Calculating the peak outflow, as opposed to total volumes, is particularly useful in urban hydrology, as it allows for the *design* of systems to accommodate a given flow rate. The Rational Method is therefore often used in the design of combined sewer overflow structures, stormwater settling tanks (*NL: bergbezinkbasins*), and other *stedelijke kunstwerken* (urban hydraulic structures).  

One of the key challenges in applying the Rational Method is determining the runoff coefficient $c_{r}$. The runoff coefficient is largely analogous to the *total imperviousness* of an area: for example, if an area is **80% impervious**, a $c_{r}$ value of **0.8** is used.  

When the peak outflow of a larger catchment (e.g., an urban drainage network) is of interest, a weighted average of the various areas within the catchment can be used:

$$
c_{r} = \frac{\sum{c_{i} \cdot A_{i}}}{\sum{A_{i}}}
$$ (eq:weighted_runoff_coefficient)

with:

|   |   |   |  
| --- | --- | --- |  
| $c_{r}$ | Weighted average runoff coefficient | [-] |  
| $c_{i}$ | Runoff coefficient of the $i^{th}$ sub-catchment | [-] |  
| $A_{i}$ | Area of the $i^{th}$ sub-catchment | $L^2$ |  

What this method does is summarize the characteristics of the entire catchment into a single value.  

Instead of estimating the total imperviousness of an area, pre-determined runoff coefficients for different *urban fabrics* (e.g., commercial areas, residential areas, etc.) can be used to construct the weighted average.  

### Hydrograph Super-position

As mentioned before, taking the weighted average of the runoff coefficients allows for the modeling of the entire catchment as one. But why is this possible? Thanks to a phenomenon known as *superposition*.  

Superposition is the process where we take individual contributions (*hydrographs*) from various sub-catchments and add them into a single hydrograph. Consider the following conceptualization of a catchment: a branched sewer network.  

### Time of Concentration  

In the previous section, we showed that the peak of the superposition-based hydrograph is equal to the outcome of the Rational Method. However, this is not always the case. To assess the applicability of the Rational Method in calculating peak runoff given a certain rainfall input, we need one more concept: *Time of Concentration*.  

By definition, the time of concentration ($ToC$, [$T$]) is the duration of the longest path to the outflow ($T_{longest}$, [$T$]) minus the shortest path ($T_{shortest}$, [$T$]):

$$
ToC = T_{longest} - T_{shortest}
$$ (eq:time_of_concentration)

If we consider the example from the previous section, the longest path (*A-outfall*) takes **7 minutes**, while the shortest path (*D-outfall*) takes only **1 minute**. This means that the ToC is **6 minutes**.  

We use the ToC to determine if the peak outflow can be calculated using the Rational Method. For this to be valid, the following condition must be met:  

$$
ToC \geq T_{rain}
$$ (eq:toc_condition)  

where $T_{rain}$ is the duration of the rainfall event used to determine the peak outflow.  

This condition is derived from the key principle of hydrograph superposition: the hydrographs can only *all* be superimposed at the outflow if the respective peaks arrive at the same time.  

To illustrate this, we will model three scenarios: rainfall intensities of **10 mm/hr**, lasting for **5, 6, and 7 minutes**, respectively.  

From the results, you can see that the 5min rainfall event case <span style="color: red;">we have to finalise this based on the code</span>.

The concept of time of concentration is not only useful to assess if the Rational Method can be applied, but it can also be used to determine the critical rainfall duration.  

Thinking back to the **IDF curves** <span style="color: red;">ref back to these curves, I assume they were discussed in the rainfall section</span>, shorter rainfall events have a higher intensity. However, if the intensities are below the time of concentration, the resulting peak flow is not necessarily higher compared to a less intense but longer-duration rainfall event, where all the individual hydrographs in the system could be superimposed.  

Having an idea of the travel times of water through a catchment is therefore a key starting point when designing the relevant outflow points. The typical velocity in a sewer network is **1 m/s**.


<!-- ```{exercise-start}
:label: ex_rational1
```

Several urban residential lots are the first drainage basin of a storm drain system. The 10-year time of concentration $t_c$, and runoff $Q$ are needed. The lots are small and it is assumed that runoff will be overland sheet flow. The project location data contains the following:

- Basin area *A* = 0.58 ha
- Length of overland flow *L* = 50 m
- Basin slope *s* = 0.02 m 
- Development density = 20 housing units per ha
- The basin is located in a village in the south of the Netherlands.

a) Compute the time of concentration and the corresponding rainfall intensity by hand. 

b) Compute the time of concentration and the corresponding rainfall intensity using Python to loop through the iterations. Use the function `glo_distribution` from the code of {numref}`fig-rainfall-duration` to compute the rainfall intensity.

c) Compute the storm runoff corresponding to the 10-year recurrence interval.

```{exercise-end}
```

:::{dropdown} Answer&nbsp;{ref}`ex_rational1`a
From {numref}`tab:3`, Manning's roughness coefficient $n$ is 0.240 (for a rural residential area).

A time of concentration $t_c$ is assumed, and the corresponding rainfall intensity $i$ is determined from the precipitation duration-recurrence interval curve in {numref}`fig-rainfall-duration`, for a 10-year recurrence interval. Values from this curve are used in the kinematic wave equation ({eq}`kinematic`) to calculate the $t_c$, and the calculated $t_c$ is compared to the assumed $t_c$. If the calculated and assumed $t_c$ values are the same, then the solution for $t_c$ has been found. If the calculated and assumed $t_c$ values are different, a new $t_c$ and corresponding $i$ are used in Equation {eq}`kinematic`. This procedure is repeated until the calculated $t_c$ is in agreement with the assumed $t_c$, as follows:

**Iteration 1**: Assuming $t_c$ = 10 minutes, $i$ = 16 mm/10 min $= 16 * \frac{60}{10} = 16*6 = 90$ mm/h. Calculated $t_c$ = 16.4 minutes.

$$
    t_c = \frac{6.92}{90^{0.4}} (\frac{0.240 * 50}{\sqrt{0.02}})^{0.6} = 16.4 \text{ min}
$$

Assumed and calculated $t_c$ are not the same. Perform a second iteration.

**Iteration 2**: Assuming $t_c$ = 16.4 minutes, $i$ = 19 mm/16.4 min = $19*\frac{60}{16.4} = 20*3.66 = 69.5$ mm/h. Calculated $t_c$ = 18.2 minutes.

$$
    t_c = \frac{6.92}{69.5^{0.4}} (\frac{0.240 * 50}{\sqrt{0.02}})^{0.6} = 18 \text{ min}
$$

Assumed and calculated $t_c$ are not the same. Perform a third iteration.

**Iteration 3**: Assuming $t_c$ = 18.2 minutes, $i$ = 20 mm/18.2 min = $20*\frac{60}{18.2} = 21*3.37 = 65.9$ mm/h. Calculated $t_c$ = 18.6 minutes.

$$
    t_c = \frac{6.92}{70.8^{0.4}} (\frac{0.240 * 50}{\sqrt{0.02}})^{0.6} = 18.6 \text{ min}
$$

Since the assumed and calculated $t_c$ are in agreement (considering a margin for reading errors), $t_c$ = 18.5 minutes and $i$ = 66 mm/h.
::: -->

<!-- :::{dropdown} Answer&nbsp;{ref}`ex_rational1`b
The results are computed in the Python code below:
::: -->

In [3]:
# import numpy as np
# import matplotlib.ticker as tick
# import matplotlib.pyplot as plt


# # Rainfall duration curve
# def f_kappa(D, T):
#     '''Only for 10 <= D <= 720 min'''
#     if D > 90 and D <= 720 and T > 120:
#         kappa = -0.310-0.0544*np.log10(D) + 0.0288*(np.log10(D))**2
#     else:
#         kappa = -0.0336 - 0.264*np.log10(D) + 0.0636*(np.log10(D))**2
#     return kappa

# def f_xi(D):
#     '''Only for 10 <= D <= 720 min'''
#     return 1.02*(7.339+0.848*np.log10(D)+2.844*(np.log10(D))**2)

# def f_gamma(D):
#     '''Only for 10 <= D <= 720 min'''
#     if D <= 104:
#         gamma = 0.04704 + 0.1979*np.log10(D) - 0.05729*(np.log10(D))**2
#     else:
#         gamma = 0.2801 - 0.0333*np.log10(D)
#     return gamma

# def glo_distribution(D, T): # use the generalized logistic distribution
#     '''Only for 10 <= D <= 720 min'''
#     kappa = f_kappa(D, T)
#     xi = f_xi(D)
#     gamma = f_gamma(D)
#     if kappa != 0:
#         #F = (1 + (1 - kappa*((D - xi)/gamma*xi)**(1/kappa)))**(-1)
#         F = xi*(1+gamma/kappa*(1-((1-np.exp(-1/T))/np.exp(-1/T))**kappa))
#     else:
#         #F = (1 + np.exp(-1*((D - xi)/gamma*xi)))**(-1)
#         F = xi*(1-gamma*np.log((1-np.exp(-1/T))/np.exp(-1/T)))
#     return F

# tc_a = 10       # assumed time of concentration
# i = 16*6        # convert from mm/10min to mm/h
# tc_i = 0        # iterated time of concentration
# n = 0.240       # mannings roughness coefficient
# Ku = 6.92
# s = 0.02        # slope
# l = 50          # length of flow segment

# for j in range(20):     # maximum of 20 iterations
#     tc_i = Ku/i**0.4 * (n*l/np.sqrt(s))**0.6
#     i = glo_distribution(tc_i, 10)*60/tc_i
#     if np.abs(tc_a - tc_i) < 0.01:
#         break
#     tc_a = tc_i
    
# print(f'The time of concentration is {tc_i:.2f} min and the corresponding rainfall intensity is {i:.2f} mm/h')

<!-- :::{dropdown} Answer&nbsp;{ref}`ex_rational1`c
Use of the precipitation duration-recurrence interval curve shown in {numref}`fig-rainfall-duration`.

From {numref}`tab:2`: $Cf$ = 1.0 for a 10-year recurrence interval.

From {numref}`tab:3`: $C$ = 0.55 (rolling terrain, normal residential)

The storm runoff is calculated using the rational method (equation {eq}`rational`) as follows:

$$
    Q_{10} = (1.0) (0.55) (66 * \frac{10^{-3}}{3600}) (0.58 * 10^4) = 0.06 \ \text{m$^3$/s}
$$
::: -->

In [4]:
# Note that the code cells below is used for the website only.

<!-- ```{exercise-start}
:label: ex_rational2
```
Drainage from the basin shown in {numref}`fig:example2` flows to a highway culvert crossing. Storm runoff is needed for a 50-year design storm using the rational method (Equation {eq}`rational`). A runoff coefficient adjustment factor $C_f$ is needed because storm runoff is to be determined for a 50-year storm. From {numref}`tab:2`, the adjustment factor is: $C_f$ = 1.2.


```{figure} ../images/figarea.png 
--- 
height: 300px 
name: fig:example2
--- 
Drainage basin in Limburg, the Netherlands
```
a) Determine the composite runoff coefficient $C$.

The rainfall intensity $i$ is needed. The 50-year precipitation duration curve (see {numref}`fig-rainfall-duration`) will be used. The time of concentration $t_c$ must first be estimated to obtain the rainfall intensity. For this example, the drainage path used to determine the time of concentration is composed of two segments. The first segment is 95 meters long and it is assumed to be overland sheet flow. The remaining 275 meter long segment is assumed to be shallow concentrated flow.

b) Compute the time of concentration for the area, which consists of the travel time for the overland sheet flow segment and the travel time for the shallow concentrated flow segment.

c) Determine the rainfall intensity corresponding to the computed time of concentration

d) Compute the storm runoff

```{exercise-end}
```

:::{dropdown} Answer&nbsp;{ref}`ex_rational2`a
A composite runoff coefficient $C$ is needed because the drainage basin contains sub-areas with different $C$ values. Sub-area $C$ values are from {numref}`tab:1`, and the composite $C$ value is calculated as follows:
 
| Description | $C$ Value | Area [ha] | Impervious Area <br /> $C_i$ $A_i$ [ha] |
| --- | --- | --- | --- | 
| Rolling Forest | 0.15 | 1.4 | 0.21 |
| Flat Light Residential | 0.35 | 1.2 | 0.42 |
| Flat Pasture | 0.25 | 2.1 | 0.53 |
| | | | 
| | Total | 4.7 | 1.16 |


The composite runoff coefficient is calculated using Equation {eq}`composite` as follows:

$$
    C = \frac{(0.21 + 0.42 + 0.53)}{4.7} = \frac{1.16}{4.7} = 0.25 [-]
$$

The area $A$ is also calculated during this step: $A$ = 4.7 hectares.
:::

:::{dropdown} Answer&nbsp;{ref}`ex_rational2`b
- **Overland sheet flow segment -** The travel time for the overland sheet flow segment is calculated as follows.

From {numref}`tab:3`: n = 0.40 (woodland and forest)

From Equation {eq}`kinematic`:

$$
    t_{osf} = \frac{6.92}{i^{0.4}} (\frac{0.40 * 95}{\sqrt{0.05}})^{0.6}
$$

From the 50-year precipitation duration-recurrence interval curve, and using a trial and error solution (see previous example problem): $t_{osf}$ = 25 minutes, with $i$ = 86 mm/h.

- **Shallow concentrated flow segment -** The travel time for the shallow concentrated flow segment is calculated as follows. 

From the location data, 140-95 = 45 m of the drainage path is over forested land with a 5 percent slope: $l$ = 45 m, and $s$ = 5 \%

From {numref}`fig:shallowconflow`: $v$ = 0.17 m/s 

From Equation {eq}`shallowflow`, the travel time for shallow concentrated flow over forested land is:

$$
    t_{scf, \text{\ forested land}} = \frac{45}{0.17} = 265 \ \text{s} = \frac{265}{60} = 4.4 \ \text{min}
$$

From the location data, the last 225 meters of the drainage path is a grassed waterway at a 1 percent slope: $l$ = 225 m, s = 1 \%

From {numref}`fig:shallowconflow`: $v$ = 0.45 m/s

The travel time for shallow concentrated flow down the grassed waterway is:

$$
    t_{scf, \text{\ grassed waterway}} = \frac{225}{0.45} = 500 \ \text{s} = \frac{500}{60} = 8.3 \ \text{min}
$$

From Equation {eq}`timeofconcentration`, the time of concentration is $t_c = 25 + 4.4 + 8.3 = 38$ min
:::

:::{dropdown} Answer&nbsp;{ref}`ex_rational2`c
From the precipitation duration-recurrence interval curve using a rainfall duration that corresponds to the 38 minute time of concentration, the amount of rainfall is approximately 42 mm, so $i = 42 * \frac{60}{38} = 66$ mm/h
:::

:::{dropdown} Answer&nbsp;{ref}`ex_rational2`d
From equation {eq}`rational`, the storm runoff is calculated as follows:

$$
    Q_{50} = (1.2) (0.25) (66 * \frac{10^{-3}}{3600}) (4.7 * 10^4) = 0.26 \ \text{m$^3$/s}
$$
::: -->

## Mass-balance modeling

(urban_evap_model)=
## Evaporation modeling